#### Setting environments

In [1]:
import torch, os, gc
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score,\
f1_score, precision_score, recall_score, roc_auc_score, average_precision_score

In [2]:
# Set options
embed_ver = ["clstm", "t5"]
test_ver = ["C018", "C039", "C035", "C033", "C061"]

# model options
data_path = "../data/test_exam/"
model_path = f"../models/cls/"
result_path = f"../results/"
os.makedirs(result_path + "prd-indiv/", exist_ok=True)

col_str = ['file_id', 'organism', 'locus_tag', 'ess']

batch_size = 256

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Define function to record perfomance result
def record_perform(emb_ver, file_id, organ, y_real, y_conf, y_prd):    
    if file_id != "O046":
        auc_roc = [roc_auc_score(y_real, y_conf)]
        auc_pr = [average_precision_score(y_real, y_conf)]
    else:
        auc_roc = None
        auc_pr = None
    
    tn, fp, fn, tp = confusion_matrix(y_real, y_prd).ravel()
    
    result = pd.DataFrame({
        "emb": [emb_ver],
        "file": [file_id],
        "organism": [organ],
        "tp": [tp],
        "fp": [fp],
        "tn": [tn],
        "fn": [fn],
        "mcc": [matthews_corrcoef(y_real, y_prd)],
        "acc": [accuracy_score(y_real, y_prd)],
        "f1": [f1_score(y_real, y_prd)],
        "prc": [precision_score(y_real, y_prd)],
        "rec": [recall_score(y_real, y_prd)],
        "npv": [precision_score(1 - y_real, 1 - y_prd)],
        "tnr": [recall_score(1 - y_real, 1 - y_prd)],
        "auc-roc": auc_roc,
        "auc-pr": auc_pr
    })

    return result


In [5]:
# Set model architecture
class Classifier(nn.Module):
    def __init__(self, input_size, num_layers, unit_decrease):
        super(Classifier, self).__init__()
        layers = [nn.BatchNorm1d(input_size), nn.Dropout(0.5)]
        in_dim = input_size
        out_dim = 1024
        for i in range(num_layers):            
            out_dim = max(2, out_dim // unit_decrease)
            layers.append(nn.Linear(in_dim, out_dim))
            self.initialize_weights(layers[-1])
            layers.append(nn.GELU())
            in_dim = out_dim
        layers.append(nn.Linear(out_dim, 1))
        self.cls_block = nn.Sequential(*layers)
        
    def initialize_weights(self, layer):
        nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='linear')
        if layer.bias is not None:
            nn.init.zeros_(layer.bias)
    
    def forward(self, x):
        return self.cls_block(x)

#### Evaluate model

In [6]:
df_eval = pd.DataFrame()

for ver in embed_ver:
    print(f"\n>>>> {ver} <<<<")

    total_label = []
    total_conf = []
    total_cls = []
    for ts_ver in test_ver:    
        # load dataset
        data = pd.read_csv(data_path + f"data-emb_{ver}-{ts_ver}_ts.csv")
        display("Raw data:", data)

        #### Preprocess for test dataset ####
        col_num = [col for col in data.columns if col not in col_str]

        # split info.& inputs & labels of the test datasets
        info_ts = data[col_str]
        X_ts = torch.tensor(data[col_num].astype('float32').values)
        y_ts = torch.tensor(data['ess'].astype('float32').values)
        print(f"Splited test dataset({ts_ver}):", X_ts.shape, y_ts.shape)
        
        # generate dataloader by the test datasets
        dataset_ts = TensorDataset(X_ts, y_ts)
        test_loader = DataLoader(dataset_ts, batch_size=batch_size, shuffle=False)

        ## Test model ##    
        # generate model instance
        model = Classifier(
            input_size=X_ts.shape[-1],
            num_layers=3,
            unit_decrease=2
        ).to(device)

        # load model weight
        model.load_state_dict(torch.load(f"{model_path}{ver}-{ts_ver}.pt", map_location=device))
        model.eval()

        ## model evaluations by test dataset ##
        results = []
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                # prediction
                preds = model(X_batch).view(-1).cpu()
                # gather the predictions
                results.append(preds)
            
        results = torch.cat(results, dim=0)
        
        # convert logits to confidences & classes
        prd_conf = torch.sigmoid(results)
        prd_cls = (prd_conf >= 0.5).int().tolist()

        # save the model prediction result
        df_pred = pd.concat([info_ts, pd.DataFrame({"conf": prd_conf.tolist()})], axis=1)
        df_pred.to_csv(f"{result_path}prd-indiv/{ver}-{ts_ver}.csv", index=False)
        
        # gather the results
        total_label.extend(df_pred['ess'].tolist())
        total_conf.extend(df_pred['conf'].tolist())
        total_cls.extend(prd_cls)
        
        # get evaluation row by testset
        eval_ts = record_perform(
            emb_ver=ver,
            file_id=ts_ver,
            organ=ts_ver,
            y_real=df_pred['ess'].to_numpy(),
            y_conf=df_pred['conf'].to_numpy(),
            y_prd=np.array(prd_cls)
        )
        df_eval = pd.concat([df_eval, eval_ts], ignore_index=True)
        print(f"- Test in {ts_ver} was done.")
    
    del model
    gc.collect()
    torch.cuda.empty_cache()

    # get total mean row
    eval_ts = record_perform(
        emb_ver=ver,
        file_id="total",
        organ="all",
        y_real=np.array(total_label),
        y_conf=np.array(total_conf),
        y_prd=np.array(total_cls)
    )
    df_eval = pd.concat([df_eval, eval_ts], ignore_index=True)
    print(f"- Test in total testset was done.")

# save the model perfomance result
df_eval.to_csv(f"{result_path}eval-indiv_cls-strain.csv", index=False)
display("Model performance:", df_eval)


>>>> clstm <<<<


'Raw data:'

,file_id,organism,locus_tag,ess,0,1,2,3,4,5,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,C018,Escherichia coli K-12 BW25113,BW25113_0001,0,-1.141337,0.000117,0.074418,-0.000446,-0.002198,-0.003431,...,-0.000123,0.004792,0.000471,0.126904,0.006709,0.013690,-0.021874,-0.049524,0.108785,-0.003517
1,C018,Escherichia coli K-12 BW25113,BW25113_0002,0,0.137700,0.000258,0.067561,-0.000478,-0.000678,-0.003556,...,-0.000045,0.001738,0.002048,0.126786,0.007048,0.008776,0.114241,-0.035615,0.105729,0.001858
2,C018,Escherichia coli K-12 BW25113,BW25113_0003,0,0.128075,0.000037,0.062052,-0.004017,-0.002748,-0.000468,...,-0.000051,0.001672,0.002281,0.126932,0.007019,0.007990,0.115545,-0.033727,0.102622,0.002154
3,C018,Escherichia coli K-12 BW25113,BW25113_0004,0,0.487113,0.000235,0.064557,-0.001244,-0.000863,-0.002970,...,-0.000038,0.001291,0.000991,0.127128,0.007119,0.008230,0.117134,-0.032810,0.105667,0.001789
4,C018,Escherichia coli K-12 BW25113,BW25113_0005,0,-0.501022,0.000158,0.070695,-0.000914,-0.001795,-0.003404,...,-0.000019,-0.000853,0.003466,0.126817,0.007367,0.006851,0.123346,-0.026789,0.104581,0.004456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4308,C018,Escherichia coli K-12 BW25113,BW25113_4702,0,-0.824822,0.000154,0.070713,-0.000348,-0.002115,-0.004313,...,-0.000115,0.004844,0.001291,0.126999,0.006747,0.013313,-0.021632,-0.051901,0.107055,-0.004071
4309,C018,Escherichia coli K-12 BW25113,BW25113_4703,0,-1.325105,0.000108,0.077958,-0.000324,-0.002058,-0.002888,...,-0.000106,0.004112,0.002155,0.127117,0.006835,0.011485,0.009256,-0.044031,0.106437,-0.002243
4310,C018,Escherichia coli K-12 BW25113,BW25113_4705,0,-1.137151,0.000133,0.076093,-0.000250,-0.001809,-0.003093,...,-0.000059,0.002871,0.002066,0.127147,0.006998,0.009537,0.041613,-0.044984,0.104478,-0.000265
4311,C018,Escherichia coli K-12 BW25113,BW25113_4706,0,-1.110526,0.000127,0.074501,-0.000332,-0.002103,-0.003567,...,-0.000113,0.004755,0.000375,0.126987,0.006722,0.013741,-0.018794,-0.051945,0.108058,-0.003450


Splited test dataset(C018): torch.Size([4313, 1024]) torch.Size([4313])
- Test in C018 was done.


'Raw data:'

...

'Raw data:'

,file_id,organism,locus_tag,ess,0,1,2,3,4,5,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,C061,Brevundimonas subvibrioides ATCC 15264,Bresu_0001,1,-0.025382,-0.054158,-0.041020,0.025128,-0.022723,-0.022706,...,-0.005814,-0.028534,-0.036309,-0.044612,-0.012766,0.044037,-0.005584,-0.023319,-0.024667,0.015848
1,C061,Brevundimonas subvibrioides ATCC 15264,Bresu_0002,1,-0.018260,-0.021806,-0.000753,0.003539,-0.033552,0.010649,...,0.050197,0.036141,-0.020034,-0.039409,0.030300,0.007444,0.038922,0.000095,0.024009,0.066186
2,C061,Brevundimonas subvibrioides ATCC 15264,Bresu_0003,0,0.006159,0.021244,0.028599,0.034836,0.013581,-0.076633,...,0.017829,-0.071718,0.027697,-0.011705,-0.032012,0.071740,-0.017354,0.017639,-0.000121,0.024740
3,C061,Brevundimonas subvibrioides ATCC 15264,Bresu_0004,0,0.007004,-0.014530,0.010058,0.001008,-0.046316,0.033357,...,-0.004594,-0.042138,-0.019864,-0.048953,-0.057098,0.028646,0.039517,-0.022908,0.023727,0.008576
4,C061,Brevundimonas subvibrioides ATCC 15264,Bresu_0005,0,0.024031,0.042662,-0.003146,0.069239,-0.064750,-0.000631,...,-0.034455,-0.005914,-0.011770,-0.015676,-0.028455,-0.037863,-0.001682,-0.012595,-0.001026,0.016611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3322,C061,Brevundimonas subvibrioides ATCC 15264,Bresu_3333,0,0.001518,0.037248,0.022039,-0.012143,0.010487,0.013753,...,-0.005680,0.012004,0.037781,-0.027722,0.031395,0.001208,-0.048559,-0.011862,0.040373,-0.008763
3323,C061,Brevundimonas subvibrioides ATCC 15264,Bresu_3334,0,-0.008089,-0.027680,-0.016765,0.012398,0.039769,-0.032880,...,-0.019672,-0.026439,0.006928,-0.045812,-0.057364,0.025876,0.050790,0.003421,-0.016292,-0.007993
3324,C061,Brevundimonas subvibrioides ATCC 15264,Bresu_3335,0,0.057358,0.024541,0.019398,-0.049149,-0.003834,0.046475,...,-0.015658,0.012642,0.041163,-0.049470,-0.015128,-0.009704,-0.014906,-0.017245,0.044353,-0.025788
3325,C061,Brevundimonas subvibrioides ATCC 15264,Bresu_3336,0,0.042842,-0.016897,-0.009269,0.021328,0.008204,0.009895,...,-0.043496,-0.033051,-0.001590,-0.045391,0.065838,-0.040051,0.016861,-0.024124,0.035798,0.011222


Splited test dataset(C061): torch.Size([3327, 1024]) torch.Size([3327])
- Test in C061 was done.
- Test in total testset was done.


'Model performance:'

,emb,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,clstm,C018,C018,233,229,3786,65,0.594411,0.931834,0.613158,0.504329,0.781879,0.983121,0.942964,0.910965,0.586572
1,clstm,C039,C039,202,338,4881,149,0.419433,0.912567,0.453423,0.374074,0.575499,0.970378,0.935237,0.831878,0.479215
2,clstm,C035,C035,239,270,3136,241,0.408286,0.868502,0.483316,0.469548,0.497917,0.928635,0.920728,0.811394,0.515062
3,clstm,C033,C033,313,347,3622,41,0.607401,0.910248,0.617357,0.474242,0.884181,0.988807,0.912572,0.949381,0.775461
4,clstm,C061,C061,216,224,2689,198,0.433438,0.873159,0.505855,0.490909,0.521739,0.931417,0.923103,0.801258,0.503849
5,clstm,total,all,1203,1408,18114,694,0.488073,0.901863,0.533718,0.460743,0.634159,0.963101,0.927876,0.848067,0.540620
6,t5,C018,C018,269,372,3643,29,0.577528,0.907025,0.572950,0.419657,0.902685,0.992102,0.907347,0.947603,0.696499
7,t5,C039,C039,327,414,4805,24,0.609819,0.921364,0.598901,0.441296,0.931624,0.995030,0.920674,0.976866,0.818725
8,t5,C035,C035,415,341,3065,65,0.635430,0.895522,0.671521,0.548942,0.864583,0.979233,0.899883,0.946234,0.780424
9,t5,C033,C033,332,325,3644,22,0.653764,0.919732,0.656775,0.505327,0.937853,0.993999,0.918115,0.968341,0.863530
